In [44]:
import datetime
datetime.datetime.utcfromtimestamp(1700000000)

datetime.datetime(2023, 11, 14, 22, 13, 20)

In [3]:
# manually remove last K dicts 
K = 6

with open(r"D:\WORKSPACE\dotabet\data\1pro_games_tmp.json", 'r', encoding='utf-8') as file:
        data = file.read()
        data = '[' + data[:-1] + ']'
        data = json.loads(data)

with open('../../data/1pro_games_AAA.json', 'w') as file:
    json.dump(data[:-K], file)

In [4]:
# CAST [{},{}] to {},{},
with open(r"D:\WORKSPACE\dotabet\data\1pro_games_tmp.json", 'r') as file:
    data = file.read()

data = data[1:]
data = data[:-1] + ','

with open('../../data/1pro_games_new.json', 'w') as file:
    file.write(data)


In [ ]:
# MERGE (without very last match, to keep tmp file non-empty)
import dotabet
file_to_merge = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"
tmp_file = r"D:\WORKSPACE\dotabet\data\1pro_games_tmp.json"

dotabet.utils.merge_fetched_data(tmp_file, file_to_merge)

In [1]:
# CHECK UNIQUE
import json
file_path = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"

with open(file_path, 'r', encoding='utf-8') as file:
        if 'tmp' in file_path:
            data = file.read()
            data = '[' + data[:-1] + ']'
            data = json.loads(data)
        else:
            data = json.load(file)
    
mids = [m['match_id'] for m in data]
len(mids), len(set(mids)) 
if len(mids) == len(set(mids)):
    print("✅")
else:
    print(f"❌ {len(mids)=} {len(set(mids))=}")

✅


In [6]:
# CHECK OVERLAP
import json
file_path1 = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"
file_path2 = r"D:\WORKSPACE\dotabet\data\1pro_games_tmp.json"

with open(file_path1, 'r', encoding='utf-8') as file:
        data1 = json.load(file)

with open(file_path2, 'r', encoding='utf-8') as file:
    if 'tmp' in file_path2:
        data2 = file.read()
        data2 = '[' + data2[:-1] + ']'
        data2 = json.loads(data2)
    else:
        data2 = json.load(file)
    
mids1 = [m['match_id'] for m in data1]
mids2 = [m['match_id'] for m in data2]
print(f"overlap: {len(set(mids1) & set(mids2))}")

overlap: 0


In [ ]:
# FILTER (REMOVE DUPLICATS)
import json
file_to_modify = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"

with open(file_to_modify, 'r', encoding='utf-8') as file:
        data = json.load(file)

mids = []
modified_data = []
c=0
for d in data:
    if d.get('match_id') not in mids:
        modified_data.append(d)
        mids.append(d.get('match_id'))
    else:
        c+=1
print(f"Found {c} duplicats")
    
with open(file_to_modify, 'w') as file:
    json.dump(modified_data, file, separators=(',', ':'))

In [1]:
from dotabet.script.rating import add_rating
add_rating()

In [3]:
import pandas as pd
import dotabet

def get_team_rating(name):
    team_id = dotabet.utils.get_team_id(name)
    if team_id:
        endpoint = f"https://www.opendota.com/api/teams/{team_id}"
        rating = dotabet.fetch.fetch_data(endpoint)['rating']
        return rating


pinnacle_path = r"D:\WORKSPACE\dotabet\data\odds\pinnacle.csv"
df = pd.read_csv(pinnacle_path)
    
df['rating1'] = df['Team_1'].apply(get_team_rating)
df['rating2'] = df['Team_2'].apply(get_team_rating)
df.to_csv(pinnacle_path)

In [1]:
from dotabet.ui import send_telegram_message
from dotabet.script.pinnacle import scrape_dota2_odds   
from dotabet.script.rating import add_rating   
from dotabet.script.recommend import recommend_elo    

if __name__ == "__main__":
    token = "6056809505:AAEci0XQUkG86E0PV1P0ujabDfYqkyUHtso"
    chat_id = "333091706"
    
    # tg_msg = scrape_dota2_odds()
    add_rating()     # update pinnacle.csv with team ratings
    send_telegram_message(token, chat_id, ''.join(tg_msg))
    
    token_rec = "7109559904:AAFIoKm_jJUnzjtEUQJ5TN8ukd5Pc4Go0A0"
    chat_id_rec = "333091706"
    tg_msg_rec = recommend_elo()  # compute elo coef and make recommendation
    send_telegram_message(token_rec, chat_id_rec, ''.join(tg_msg_rec))
    


NameError: name 'tg_msg' is not defined

In [1]:
from dotabet.script.recommend import recommend_elo  
recommend_elo()

['Bet on psg quest @ 2.490(>2.176) vs gaimin gladiators Odds diff: 0.314',
 'Bet on psg quest @ 1.709(>1.544) vs heroic Odds diff: 0.165',
 'Bet on aurora @ 2.430(>2.295) vs liquid Odds diff: 0.135',
 'Bet on xtreme @ 1.478(>1.343) vs gaimin gladiators Odds diff: 0.135',
 'Bet on falcons @ 1.350(>1.235) vs azure Odds diff: 0.115',
 'Bet on liquid @ 1.400(>1.354) vs heroic Odds diff: 0.046',
 'Bet on betboom @ 1.571(>1.539) vs azure Odds diff: 0.032',
 'Bet on shopify rebellion @ 1.662(>1.649) vs natus vincere Odds diff: 0.013']

In [1]:
from dotabet.ui import send_telegram_message
from dotabet.script.pinnacle import scrape_dota2_odds    
from dotabet.script.recommend import recommend_elo    

if __name__ == "__main__":
    token = "6056809505:AAEci0XQUkG86E0PV1P0ujabDfYqkyUHtso"
    chat_id = "333091706"
    
    tg_msg = scrape_dota2_odds()
    send_telegram_message(token, chat_id, ''.join(tg_msg))
    
    token_rec = "7109559904:AAFIoKm_jJUnzjtEUQJ5TN8ukd5Pc4Go0A0"
    chat_id_rec = "333091706"
    tg_msg_rec = recommend_elo()  # compute elo coef and make recommendation
    send_telegram_message(token_rec, chat_id_rec, ''.join(tg_msg_rec))

block[0/12]=Match_date: 2024-05-24
block[1/12]=League_name: DREAMLEAGUE
❗Data already exist in CSV: 
block[2/12]=Match_odds: data=['2024-05-24', 'DREAMLEAGUE', 'azure', '1.666', 'heroic', '2.240', '(Match)', 1368.52, 1364.54, None, None]
❗Data already exist in CSV: 
block[3/12]=Match_odds: data=['2024-05-24', 'DREAMLEAGUE', 'azure', '1.719', 'heroic', '2.140', '(Map 1)', 1368.52, 1364.54, None, None]
block[4/12]=<Skip non-(Match)> 
block[5/12]=<Skip non-(Match)> 
block[6/12]=Match_date: 2024-05-25
block[7/12]=League_name: DREAMLEAGUE
❗Data already exist in CSV: 
block[8/12]=Match_odds: data=['2024-05-25', 'DREAMLEAGUE', 'gaimin gladiators', '2.830', 'falcons', '1.440', '(Match)', 1546.13, 1680.69, None, None]
❗Data already exist in CSV: 
block[9/12]=Match_odds: data=['2024-05-25', 'DREAMLEAGUE', 'gaimin gladiators', '2.470', 'falcons', '1.555', '(Map 1)', 1546.13, 1680.69, None, None]
block[10/12]=<Skip non-(Match)> 
block[11/12]=<Skip non-(Match)> 
Message sent successfully.
Message s

In [1]:
from dotabet.ui import send_telegram_message
from dotabet.script.pinnacle import scrape_dota2_odds   
from dotabet.script.recommend import recommend_elo    

token = "6056809505:AAEci0XQUkG86E0PV1P0ujabDfYqkyUHtso"
chat_id = "333091706"



# dota2_log_bot
rec_token = "7109559904:AAFIoKm_jJUnzjtEUQJ5TN8ukd5Pc4Go0A0"
tg_msg_rec = recommend_elo()  # compute elo coef and make recommendation
send_telegram_message(rec_token, chat_id, ''.join(tg_msg_rec))


Message sent successfully.


In [1]:
import argparse
from dotabet.ui import send_telegram_message
from dotabet.script.get_new import get_new_matches  
from dotabet.script.link import link  
from dotabet.script.pinnacle import scrape_dota2_odds   
from dotabet.script.recommend import recommend_elo    
import dotabet

def get_new_matches_action(chat_id, token):
    log_msg = get_new_matches()
    send_telegram_message(token, chat_id, ''.join(log_msg))

def link_action(chat_id, link_token, checkout_token):
    link_msg = link()
    send_telegram_message(link_token, chat_id, ''.join(link_msg))
    checkout_msg = dotabet.checkout.compute_elo_results()
    send_telegram_message(checkout_token, chat_id, ''.join(checkout_msg))

def scrape_odds_action(chat_id, parse_token, rec_token):
    tg_msg = scrape_dota2_odds()
    send_telegram_message(parse_token, chat_id, ''.join(tg_msg))
    tg_msg_rec = recommend_elo()
    send_telegram_message(rec_token, chat_id, ''.join(tg_msg_rec))



chat_id = '333091706'


In [2]:
log_token = '7109559904:AAFIoKm_jJUnzjtEUQJ5TN8ukd5Pc4Go0A0'
get_new_matches_action(chat_id, log_token)

Ready to fetch the Newest. Most recent id: 7768649413 (31 May 2024 10:48)
Finish 🏁 Reached match_id=7767152528
Message sent successfully.


In [4]:
link_token = '7074261020:AAG5hh_xsFvGXW7gBDnwH4ocwJzrmCGYSPY'
checkout_token = '7311473809:AAFqyhg7HIbtTu7UawrGCU5qr5v2XMmT9No'
link_action(chat_id, link_token, checkout_token)

Failed to send message.
Message sent successfully.


In [5]:
parse_token = "7416284654:AAGhRdKNgUlwDrBRPp_xRaMO151aMMh11jY"
rec_token = "7227209457:AAHHtQ2mSnHyK6zzyIjQZFyrLd4CXBZn3_o"
scrape_odds_action(chat_id, parse_token, rec_token)

block[0/10]=Match_date: 2024-05-31
block[1/10]=League_name: RIYADH MASTERS: EASTERN EUROPE QUALIFIER
❗Data already exist in CSV: 
block[2/10]=Match_odds: data=['2024-05-31', 'RIYADH MASTERS: EASTERN EUROPE QUALIFIER', '1win', '1.268', 'yellow submarine', '3.850', '(Match)', 1282.67, None, None, None]
❗Data already exist in CSV: 
block[3/10]=Match_odds: data=['2024-05-31', 'RIYADH MASTERS: EASTERN EUROPE QUALIFIER', '1win', '1.383', 'yellow submarine', '3.080', '(Map 1)', 1282.67, None, None, None]
block[4/10]=<Skip non-(Match)> 
block[5/10]=<Skip non-(Match)> 
❗Data already exist in CSV: 
block[6/10]=Match_odds: data=['2024-05-31', 'RIYADH MASTERS: EASTERN EUROPE QUALIFIER', 'natus vincere', '1.146', 'sibe', '5.750', '(Match)', 1311.13, 1187.7, None, None]
❗Data already exist in CSV: 
block[7/10]=Match_odds: data=['2024-05-31', 'RIYADH MASTERS: EASTERN EUROPE QUALIFIER', 'natus vincere', '1.268', 'sibe', '3.850', '(Map 1)', 1311.13, 1187.7, None, None]
block[8/10]=<Skip non-(Match)> 
b

In [10]:
data[0]

{'match_id': 7660501638,
 'dire_captain': 1202267677,
 'dire_name': 'Nemiga Gaming',
 'dire_team_id': 5014799,
 'game_mode': 2,
 'leagueid': 16343,
 'start_time': 1711717186,
 'lobby_type': 1,
 'metadata': None,
 'patch': 54,
 'picks_bans': [{'is_pick': False, 'hero_id': 137, 'team': 0, 'order': 0},
  {'is_pick': False, 'hero_id': 66, 'team': 1, 'order': 1},
  {'is_pick': False, 'hero_id': 98, 'team': 1, 'order': 2},
  {'is_pick': False, 'hero_id': 33, 'team': 0, 'order': 3},
  {'is_pick': False, 'hero_id': 96, 'team': 1, 'order': 4},
  {'is_pick': False, 'hero_id': 129, 'team': 1, 'order': 5},
  {'is_pick': False, 'hero_id': 49, 'team': 0, 'order': 6},
  {'is_pick': True, 'hero_id': 72, 'team': 0, 'order': 7},
  {'is_pick': True, 'hero_id': 19, 'team': 1, 'order': 8},
  {'is_pick': False, 'hero_id': 18, 'team': 0, 'order': 9},
  {'is_pick': False, 'hero_id': 48, 'team': 0, 'order': 10},
  {'is_pick': False, 'hero_id': 78, 'team': 1, 'order': 11},
  {'is_pick': True, 'hero_id': 105, 't

In [24]:
import dotabet
import os

teams_csv_path = r"D:\WORKSPACE\dotabet\constants\teams.csv"
current_ids = dotabet.utils.get_teams_composition_ids(teams_csv_path)

In [27]:
len(set(current_ids))

32

In [36]:
set(current_ids) - set(excel)

{0, 950760809, 1248263093, 1340615589, 2117989629, 2368818725}

In [70]:
import pandas as pd
from datetime import datetime
features_csv_path = r"D:\WORKSPACE\dotabet\data\top_teams\features.csv"
df = pd.read_csv(features_csv_path)
df = df.sort_values(by='start_time', ascending=True)

str_agg = lambda x: x.mode()[0] if not x.empty else None
agg_dict = {col: 'mean' for col in df.select_dtypes(include=['int', 'float']).columns}  
agg_dict.update({col: str_agg for col in df.select_dtypes(include=['object']).columns}) 

dfg = df.groupby(['match_id']) 
ndf = dfg.agg(agg_dict)
ndf=ndf.drop('match_id', axis=1)
ndf=ndf.reset_index()

In [72]:
ndf.to_csv(r"D:\WORKSPACE\dotabet\data\top_teams\lol.csv", index=False)